In [1]:
from src import Tensor,cross_entropy,SGD,Linear, Module, Sequential, ReLU,dataloader
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [3]:
class Net(Module):
    def __init__(self, inc, outc):
        super().__init__()
        self.seq = Sequential(
            Linear(inc, 64, bias=True),
            ReLU(),
            Linear(64, 64*2, bias=True),
            ReLU(),
            Linear(64*2, outc),
        )

    def forward(self, x):
        x = self.seq(x)
        return x


net = Net(train_X.shape[1], len(np.unique(train_y)))
optimizer = SGD(net.trainable_params(), lr=0.001, momentum=0.9)
for epoch in range(10):  # loop over the dataset multiple times
    losses = []
    accs = []
    for X, y in dataloader(64*2, train_X, train_y):

        # forward + backward + optimize
        outputs = net.forward(X)
        loss = cross_entropy(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        assert loss.item() > 0, loss
        optimizer.step()

        losses.append(loss.item())
        accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
    print(
        f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')

print('Finished Training')

[Epoch   1] loss: 6.960 accuracy: 7.03
[Epoch   2] loss: 4.084 accuracy: 19.88
[Epoch   3] loss: 2.861 accuracy: 38.35
[Epoch   4] loss: 2.267 accuracy: 45.11
[Epoch   5] loss: 1.799 accuracy: 53.58
[Epoch   6] loss: 1.498 accuracy: 58.19
[Epoch   7] loss: 1.288 accuracy: 63.79
[Epoch   8] loss: 1.074 accuracy: 69.50
[Epoch   9] loss: 0.983 accuracy: 73.09
[Epoch  10] loss: 0.907 accuracy: 73.38
Finished Training
